## Minerva Grade Opoortunities
### Nicolas A Gort Freitas

In [587]:
import requests
import json
import urllib.parse, urllib.request, urllib.error
import numpy as np
import pandas as pd

np.set_printoptions(suppress=True)

In [ ]:
def retrieve_parallel(key,key2,var): #find keys at the same level in nested dictionaries
    if hasattr(var,'items'):
        for k, v in var.items():
            if k == key:
                try: 
                    yield var[key2]
                except: 
                    pass
                
            if isinstance(v, dict):
                for result in retrieve_parallel(key,key2, v):
                    if type(result)==int or type(result)==str:
                        yield result
            elif isinstance(v, list):
                for d in v:
                    for result in retrieve_parallel(key,key2, d):
                        yield result

In [558]:
lo = requests.get('https://seminar.minerva.kgi.edu/api/v1/hc-index-items?outcomeType=lo',
                headers={'Authorization':'Token ****************************************'}).json()

In [559]:
hc = requests.get('https://seminar.minerva.kgi.edu/api/v1/hc-index-items',
                headers={'Authorization':'Token ****************************************'}).json()

In [560]:
hctree = requests.get('https://seminar.minerva.kgi.edu/api/v1/hc-trees/current?tree',
                headers={'Authorization':'Token ****************************************'}).json()

In [562]:
hc_index = {}
hc_indices = list(retrieve_parallel('hashtag','id',hctree))
hc_names = list(retrieve_parallel('id','hashtag',hctree))

for i,j in zip(hc_indices,hc_names):
    hc_index[i]=j

In [563]:
def logrades(n):
    return requests.get('https://seminar.minerva.kgi.edu/api/v1/outcome-assessments-with-assessed?learning-outcome='+str(n),
                headers={'Authorization':'Token ****************************************'}).json()

def hcgrades(n):
    return requests.get('https://seminar.minerva.kgi.edu/api/v1/outcome-assessments-with-assessed?hc-item='+str(n),
                headers={'Authorization':'Token ****************************************'}).json()



In [564]:
def hc_numgrades(hcid):
    hcvar = hcgrades(hcid)
    weights = []
    for i,j in enumerate(hcvar):
        if j['score']==0.0:
            weights.append(0.0)
        elif j['assignment']==None:
            weights.append(1.0)
        elif 'weight' in j['assignment'].keys():
            weights.append(j['assignment']['weight'])
        else:
            weights.append(weights[-1])

    return sum(weights)

In [707]:
def hc_mean(hcid):
    for i in hc:
        if i['hc-item']==int(hcid):
            if i['mean']==None:
                return np.nan
            else:
                return i['mean']

In [938]:
def impact4(x):
    hcimpact = ((x.Mean*x.N+4)/(x.N+1))
    if x.N>0:
        return (x.Mean_course*x.HC_count - x.Mean + hcimpact)/(x.HC_count)-x.Mean_course
    if x.N==0:
        #print ('a')
        return (x.Mean_course*x.HC_count+4)/(x.HC_count + 1) - x.Mean_course

In [575]:
means = np.array([hc_mean(x) for x in hc_index.keys()])
ngrades = np.array([hc_numgrades(x) for x in hc_index.keys()])

In [711]:
cournerstones = list(retrieve_parallel('hashtag','cornerstone-code',hctree))

In [780]:
hc_data = pd.DataFrame({
'HC' : list(hc_index.values()), 
    'Mean' : means,
    'N' : ngrades,
    'Course' : cournerstones
    #'potential4' : potential4,
    #'potential3' : potential3
})

In [839]:
hc_data.head(2)

,Course,HC,Mean,N
0,MC,#openmind,3.750000,4.0
1,MC,#critique,3.035714,28.0


In [799]:
scores = hc_data.loc[:,['Mean','Course']].groupby('Course').agg({np.mean,lambda x: len(x[x > 0])})
scores.columns = ['Mean','HC_count']
scores

,Mean,HC_count
Course,,
CS,3.288901,25.0
EA,3.574988,24.0
FA,3.351841,21.0
MC,3.474969,22.0


In [937]:
joint = hc_data.join(scores,on='Course',rsuffix='_course')

In [936]:
hc_data['Impact_4']=joint.apply(impact4,axis=1)

#example: What HCs are most convenient to tag to improve my score in FA?
hc_data[hc_data.Course=='FA'].sort_values('Impact_4',ascending=False)
#hc_data.sort_values('Impact_4')

,Course,HC,Mean,N,Impact_4
36,FA,#gametheory,3.333333,3.0,0.007937
31,FA,#broadframing,3.000000,5.0,0.007937
32,FA,#decisiontrees,2.857143,7.0,0.006803
30,FA,#risk,3.000000,7.0,0.005952
28,FA,#sunkcost,3.500000,4.0,0.004762
56,FA,#optimization,3.285714,7.0,0.004252
29,FA,#discounting,3.250000,8.0,0.003968
14,FA,#estimation,3.277778,9.0,0.003439
18,FA,#confidenceintervals,3.300000,10.0,0.003030
21,FA,#significance,3.058824,17.0,0.002490
